# Perlin Noise SynthSeg
env: python3.8 tensorflow

In [ ]:
from SynthSegAAA.perlin_noise import randomize_perlin_label
from ext.lab2im import utils
import numpy as np

lab = utils.load_volume("data/uwa_aaa_combined/Patient1_combined.nii.gz", dtype='int', aff_ref=np.eye(4))
output = randomize_perlin_label(lab, target=0, layer=2)

Generating Perlin noise with parameters:
Grid size: (110, 104, 126)
Scale: 6.911655714027905
Octaves: 6
Persistence: 0.44694719794020094
Lacunarity: 1.9911592141651353
Threshold: 0.5798468454312964
Minimum cloud size: 39776
Seed: 96
Generating Perlin noise with parameters:
Grid size: (110, 104, 126)
Scale: 4.376846168147354
Octaves: 6
Persistence: 0.5782777869998654
Lacunarity: 2.1904969136110317
Threshold: 0.6217848634167679
Minimum cloud size: 2744
Seed: 299


In [ ]:
from utils.check_image import check_image

check_image(
    [
        {"image": output , "title": "image", "is_label": False},
    ]
)

interactive(children=(IntSlider(value=0, description='slice_index_0', max=125), Output()), _dom_classes=('widg…

In [1]:
import os
import numpy as np
import nibabel as nib
from SynthSegAAA.brain_generator import BrainGenerator

np.random.seed(42)

input_label_path = "data/uwa_aaa_combined"
image_path = "data/Dataset002_perlin_aaa/imagesTr"
label_path = "data/Dataset002_perlin_aaa/labelsTr"
os.makedirs(image_path, exist_ok=True)
os.makedirs(label_path, exist_ok=True)
label_files = os.listdir(input_label_path)
label_files.sort()

Using TensorFlow backend.


In [ ]:

from utils.nrrd2nii import nrrd2nii


for i, label_file in enumerate(label_files):
    label = nib.load(os.path.join(input_label_path, label_file))
    print(f"Processing {label_file} ({i + 1}/{len(label_files)})", label.shape)

Processing Patient10_combined.nii.gz (1/21) (88, 84, 100)
Processing Patient11_combined.nii.gz (2/21) (120, 146, 232)
Processing Patient12_combined.nii.gz (3/21) (236, 168, 246)
Processing Patient13_combined.nii.gz (4/21) (172, 124, 280)
Processing Patient14_combined.nii.gz (5/21) (198, 150, 194)
Processing Patient15_combined.nii.gz (6/21) (120, 108, 150)
Processing Patient16_combined.nii.gz (7/21) (172, 170, 250)
Processing Patient17_combined.nii.gz (8/21) (222, 136, 244)
Processing Patient18_combined.nii.gz (9/21) (126, 122, 128)
Processing Patient1_combined.nii.gz (10/21) (110, 104, 126)
Processing Patient20_combined.nii.gz (11/21) (152, 134, 218)
Processing Patient26_combined.nii.gz (12/21) (112, 110, 118)
Processing Patient32_combined.nii.gz (13/21) (96, 86, 148)
Processing Patient33_combined.nii.gz (14/21) (114, 86, 154)
Processing Patient34_combined.nii.gz (15/21) (164, 118, 154)
Processing Patient35_combined.nii.gz (16/21) (94, 108, 126)
Processing Patient3_combined.nii.gz (17/

# Generation

In [1]:
import os
import numpy as np
import nibabel as nib
from SynthSegAAA.brain_generator import BrainGenerator
from tqdm import tqdm

np.random.seed(42)

input_label_path = "data/nnunet_raw/Dataset001_raw_aaa/labelsTr"
image_path = "data/nnunet_raw/Dataset003_perlin_synthseg/imagesTr"
label_path = "data/nnunet_raw/Dataset003_perlin_synthseg/labelsTr"
os.makedirs(image_path, exist_ok=True)
os.makedirs(label_path, exist_ok=True)
label_files = os.listdir(input_label_path)
label_files.sort()
selected_num = 9
generate_times = 1
count = 0

# Calculate total iterations for progress bar
total_iterations = len(label_files[:selected_num]) * generate_times

# Add tqdm progress bar
with tqdm(total=total_iterations, desc="Generating brains") as pbar:
    for label_file in label_files[:selected_num]:
        brain_generator = BrainGenerator(
            os.path.join(input_label_path, label_file), 
            generation_labels=np.array([0, 1, 2, 3]),
            output_labels=np.array([0, 1, 2, 0]),
        )
        for i in range(generate_times):
            image, label = brain_generator.generate_brain()
            nib.save(
                nib.Nifti1Image(image, np.eye(4)),
                os.path.join(image_path, f"aaa_{count:03d}_0000.nii.gz"),
            )
            nib.save(
                nib.Nifti1Image(label, np.eye(4)),
                os.path.join(label_path, f"aaa_{count:03d}.nii.gz"),
            )
            count += 1
            pbar.update(1)

Using TensorFlow backend.
Generating brains:   0%|          | 0/9 [00:00<?, ?it/s]2025-04-16 13:45:22.118397: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2025-04-16 13:45:22.388485: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-16 13:45:22.388525: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3070 computeCapability: 8.6
coreClock: 1.755GHz coreCount: 46 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 417.29GiB/s
2025-04-16 13:45:22.389168: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2025-04-16 13:45:22.389462: W ten

# Inspection

In [6]:

from utils.check_image import check_image
import nibabel as nib
from utils.nrrd2nii import nrrd2nii

id = "000"
label = nib.load(f"{label_path}/aaa_004.nii.gz").get_fdata()
image = nib.load(f"{image_path}/aaa_004_0000.nii.gz").get_fdata()
check_image(
    [
        {"image": label, "title": "label", "is_label": True},
        {"image": image, "title": "image", "is_label": False},
    ]
)

interactive(children=(IntSlider(value=0, description='slice_index_0', max=193), IntSlider(value=0, description…